## Property

Setting `self.name` in the `__init__` allows us to exploit the setter during construction.

In [36]:
class Cat:
    def __init__(self, name):
        self.name = name
    
    @property
    def name(self):
        print("Getting name...")
        # we use _name here
        return self._name
    
    @name.setter
    def name(self, value):
        print("Setting name...")
        # we use _name here
        self._name = value

The setter is used during construction.

In [37]:
test = Cat('Top')

Setting name...


In [38]:
test.name

Getting name...


'Top'

In [39]:
test.name = "Simba"

Setting name...


In [40]:
test.name

Getting name...


'Simba'

Can still set `_name` but this isn't recommended.

In [41]:
test._name = 'Bad Cat'
test._name

'Bad Cat'

In [42]:
test.name

Getting name...


'Bad Cat'

## OOP

This shows:
* Abstract method.
    * This leaves a gap that the subclass **must** fill.
* Read-only property pattern.
    * No setter property has been defined, so more immutability.
    * Caveat: true immutability is not possible in Python.


In [43]:
from abc import abstractmethod, ABC
from textwrap import dedent
class Animal(ABC):
    def __init__(self, name):
        self.name = name
    
    @property
    @abstractmethod
    def pet(self) -> bool:
        print('Getting pet...')
        pass
    
    # @final from  Python 3.7 
    def info(self):
        info_string = f"""
        Name: {self.name}
        Pet status: {self.pet}
        """
        print(dedent(info_string))
        

In [44]:
class Dog(Animal):
    def __init__(self, name):
        super().__init__(name)
    
    # notice: the subclass implementation must also be a property
    @property
    def pet(self):
        return True

In [45]:
clifford = Dog("Clifford")

In [46]:
clifford.info()


Name: Clifford
Pet status: True



The `pet` attribute is read-only.

In [47]:
clifford.pet = False

AttributeError: can't set attribute

In [48]:
clifford.info()


Name: Clifford
Pet status: True



## Boolean filtering

* Dictionaries are incredibly versatile, even funtions can be keys.
* We can use this to create a clean matching function, similar to a Java switch.
* This is a lot cleaner than the if else alternative.
* This can be useful if we don't want any short circuiting and multiple matches are possible.

In [73]:
def match(x: int) -> list:
    bool_dict = {
        lambda num: num + 1 == 2: "That's numberwang!",
        lambda num: num % 2 == 0: "Even Stevens!",
        lambda num: num % 3 == 0: "Divisible by 6."
    }
    result = [result for test, result in bool_dict.items() if test(x)]
    return result

In [50]:
match(1)

["That's numberwang!"]

In [51]:
match(2)

['Even Stevens!']

In [52]:
match(6)

['Even Stevens!', 'Divisible by 6.']

In [53]:
match(-1)

[]

This implementation is better if we want `CASE WHEN` logic, where only the final match is kept.

In [66]:
from typing import Optional

In [67]:
def match(x: int) -> Optional[str]:
    bool_dict = {
        lambda num: num + 1 == 2: "That's numberwang!",
        lambda num: num % 2 == 0: "Even Stevens!",
        lambda num: num % 3 == 0: "Divisible by 6."
    }
    result = [result for test, result in bool_dict.items() if test(x)]
    result.insert(0, None) # insert None at the front in case of no match
    return result[-1]

In [68]:
match(1)

"That's numberwang!"

In [69]:
match(0)

'Divisible by 6.'

In [70]:
match(6)

'Divisible by 6.'

In [71]:
match(-1)

## Dynamic Class Attribute

Nice way to differentiate the result of a method call by the context.

You'll get different values depending on the class name call versus the object call.

In [38]:
from types import DynamicClassAttribute

# Metaclass
class AnimalMeta(type):

    def __getattr__(self, value):
        return AnimalMeta.value

    # Metaclasses dynprop:
    value = 'meta'

class Bird(metaclass=AnimalMeta):
    def __init__(self, in_):
        self._value = in_

    @DynamicClassAttribute
    def value(self):
        return self._value

In [39]:
Bird.value

'meta'

In [37]:
a = Bird('not meta')
print(a.value)

not meta


## Cache init args

In [28]:
def cache_init_kwargs(func):
    def saved_init(*args, **kwargs):
        arguments = inspect.signature(func).bind(*args, **kwargs).arguments
        arguments.pop('self', None)
        self = args[0]
        self._init_arguments = arguments
        return func(*args, **kwargs)
    
    return saved_init

In [29]:
class Computer:
    @cache_init_kwargs
    def __init__(self, storage: float, ram: float):
        self.storage = storage
        self.ram = ram
        
    def __repr__(self):
        return f"This computer has a storage of {self.storage}GB and {self.ram}GB ram."

In [30]:
test = Computer(500, 16)

In [31]:
test._init_arguments

OrderedDict([('storage', 500), ('ram', 16)])

## Guardrail

In [2]:
from typing import Callable
from functools import wraps
import inspect

In [17]:
def guardrail(guarded_argument: str, boolean_test: Callable, error_message: str):
    def wrapper(func: Callable) -> Callable:
        @wraps(func)
        def guardrailed_function(*args, **kwargs):
            arguments = inspect.signature(func).bind(*args, **kwargs).arguments
            value = arguments.get(guarded_argument, None)
            
            if value is None:
                raise ValueError("argument specified was not provided")
                
            assert boolean_test(value), f"arg: {value} failed guardrail test. {error_message}"
            
            return func(*args, **kwargs)
        
        return guardrailed_function
    
    return wrapper        

In [21]:
@guardrail('x', lambda x: isinstance(x, int), "must be an integer.")
def add_one(x):
    return x+1

In [22]:
add_one(1)

2

In [23]:
add_one(1.0)

AssertionError: arg: 1.0 failed guardrail test. must be an integer.